<a href="https://colab.research.google.com/github/Roger-G/Data-augmentation/blob/master/LR_ag_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# df_train = pd.read_csv('/content/drive/My Drive/Data/Back_trans/datasets/twitter/train.tsv', sep = '\t')
# print('info', df_train.info())
# print('value_counts', df_train.label.value_counts())
# ! wget http://nlp.stanford.edu/data/glove.6B.zip
# ! unzip glove.6B.zip

## Package

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import seaborn as sns
from torch import optim
import torch.nn.functional as F
import pickle
import csv
import torch.utils.data as Data
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') 
from sklearn.svm import LinearSVC
from tqdm import trange
import pandas as pd
import torchtext.vocab as Vocab
import collections
# with open("glove.6B.50d.txt", "rb") as lines:
#     w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
#            for line in lines}

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn import svm
import os.path as path
import re
from sklearn.utils import resample
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
from sklearn.pipeline import Pipeline

## processor

In [5]:
class Preprocessor():
    
  def __init__(self, data, max_seq_len, vocab):
    self.data = data
    self.max_seq_len = max_seq_len
    self.vocab = vocab
    self.PAD, self.BOS, self.EOS = '<pad>', '<bos>', '<eos>'
    self.all_tokens = []
    self.all_seqs = []

  def process_one_seq(self, seq_tokens):
    self.all_tokens.extend(seq_tokens) # add seq_tokens into one list, like extend the list, [..., seq_tokens].
    seq_tokens = [self.PAD] * (self.max_seq_len - len(seq_tokens)) + seq_tokens
    self.all_seqs.append(seq_tokens) # add seq_tokens become a list element, [...,[seq_tokens]].
      
  def build_data(self):
    if self.vocab:
      indices = [[self.vocab.stoi[w] for w in seq] for seq in self.all_seqs] # use its vocab_dic to represent the sentence. 
    else:
      tokens_dic = collections.Counter(self.all_tokens) 
      self.vocab = Vocab.Vocab(tokens_dic, specials=[self.PAD])
      indices = [[self.vocab.stoi[w] for w in seq] for seq in self.all_seqs] # use its vocab_dic to represent the sentence. 
    return indices
  
  @staticmethod
  def normalizeString(s):
    s = s.str.lower()
    s = s.str.replace(r"<br />",r" ")
    s = s.str.replace(r'(\W)(?=\1)', '')
    s = s.str.replace(r"([.!?])", r" \1")
    s = s.str.replace(r"[^a-zA-Z.!?]+", r" ")
    return s

  def read_data(self):
    df = self.data.copy()
    df = df.dropna()
    df['sentence'] = self.normalizeString(df['sentence'])
    df.reset_index(inplace=True, drop=True)
    k = len(df)
    target = []

    for line in range(k):
      in_seq_tokens = df['sentence'][line].split(' ')
      if len(in_seq_tokens) > self.max_seq_len - 1:
        continue
      self.process_one_seq(in_seq_tokens)
      target.append(df.label[line])

    in_data = self.build_data() # in_tokens is the list where contains every word, in_seqs is a list where its element are the sentence in French.

    return [in_data, target]
    # Data.TensorDataset(in_data, torch.tensor(target))

## Loader

In [6]:
class Loader():


  def path_setting(self, task_name, tech_name):
    abspath = '/content/drive/My Drive/Data/'
    model_path = path.join(abspath, tech_name + '/save_model')
    dataset_path = path.join(abspath, tech_name + '/datasets/{}/'.format(task_name))
    save_path = path.join(abspath, tech_name + '/aug_data/{}/'.format(task_name))
    
    return model_path, dataset_path, save_path

  @staticmethod
  def load_data(path_, FILE_NAME, ratio, tech_name):

    df_train = pd.read_csv(os.path.join(path_,'train.tsv'), sep='\t',  error_bad_lines=False)
    df_train = df_train.dropna()

    num = int(len(df_train) * ratio)
    df_train = df_train[ : num ]
    num_DA = len(df_train[df_train.label==0]) - len(df_train[df_train.label==2])

    if FILE_NAME == 'oversampling':
      # 选出差额， 补到原数据上面
      df_over_samp = resample(df_train[df_train.label==2], n_samples=num_DA, random_state=100, replace=True)
      df = pd.concat([df_over_samp, df_train])

    elif FILE_NAME == 'downsampling':
      df_down_samp = resample(df_train[df_train.label !=2], n_samples=len(df_train[df_train.label==2])*3, random_state=100, replace=True)
      df = pd.concat([df_down_samp, df_train[df_train.label == 2]])
    else:
      df = pd.read_csv(os.path.join(path_, FILE_NAME + '.tsv'), sep='\t',  error_bad_lines=False)    
      df = df.dropna()
      if FILE_NAME == 'train' or tech_name == 'Back_trans':
        df = df[ : num ]

    if FILE_NAME not in ['train', 'dev', 'oversampling', 'test' , 'downsampling']:
      try:
        df_over_samp = resample(df, n_samples=num_DA, replace = False, random_state=10) 
      except ValueError:
        df_extro = resample(df_train[df_train.label==2], n_samples=num_DA - len(df), replace = True, random_state=10)
        df_over_samp = pd.concat([df, df_extro])

      df = pd.concat([df_train, df_over_samp]) 
    
    else: 
      pass

    df = df.sample(frac = 1, replace=False, random_state=102).reset_index().drop(columns = ['index'])
    print('value_counts', df.label.value_counts())
    return df



In [7]:
#@title


#@



def lr_cv(splits, X, Y, pipeline, average_method):
    dev_data = pd.read_csv('/content/drive/My Drive/Data/Back_trans/aug_data/twitter/dev.tsv', sep='\t')
    dev_data.dropna(inplace=True)
    x_dev, y_dev = dev_data.sentence, dev_data.label
    kfold = StratifiedKFold(n_splits=splits, shuffle=True, random_state=777)
    accuracy = []
    precision = []
    recall = []
    f1 = []
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test],Y[test])
        
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        # print('first precision', precision)
        print('              negative    neutral     positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    prediction_dev = lr_fit.predict(x_dev)
    dev_accuracy = lr_fit.score(x_dev, y_dev)


    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))
    print('*'* 50)
    precision = precision_score(y_dev, prediction_dev, average=average_method)
    # print('precision', precision)
    recall = recall_score(y_dev, prediction_dev, average=average_method)
    f1 = f1_score(y_dev, prediction_dev, average=average_method)
    print("accuracy: %.2f" % (dev_accuracy))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))


## Evaluation （test <--> dev)

In [8]:
def eval_matrix(data, pipeline, average_method, save_path):

  dev_data = pd.read_csv(path.join(save_path, 'test.tsv'), sep='\t')
  dev_data.dropna(inplace=True)
  x_dev, y_dev = dev_data.sentence, dev_data.label
  x_train, y_train = data.sentence, data.label
  lr_fit = pipeline.fit(x_train, y_train)
  prediction_dev = lr_fit.predict(x_dev)
  dev_accuracy = lr_fit.score(x_dev, y_dev)
  precision = precision_score(y_dev, prediction_dev, average=average_method)
  recall = recall_score(y_dev, prediction_dev, average=average_method)
  f1 = f1_score(y_dev, prediction_dev, average=average_method)
  print("accuracy: %.4f" % (dev_accuracy))
  print("precision: %.4f" % (np.mean(precision)))
  print("recall: %.4f" % (np.mean(recall)))
  print("f1 score: %.4f" % (np.mean(f1)))


In [9]:

# class excuter():
def pipelines(tkz, clf):

  ROS_pipeline = make_pipeline(tkz, RandomOverSampler(random_state=777), clf)
  SMOTE_pipeline = make_pipeline(tkz, SMOTE(random_state=777), clf)
  
  original_pipeline = Pipeline([
      ('vectorizer', tkz),
      ('classifier', clf)
  ])
  return ROS_pipeline, original_pipeline

def execute(test_list, ratio, task_name, tech_name, tkz, clf):
  loader = Loader()
  model_path, dataset_path, save_path = loader.path_setting(task_name, tech_name)
  train_data = pd.read_csv(path.join(save_path, 'train.tsv'), sep='\t')
  
  ROS_pipeline, original_pipeline = pipelines(tkz, clf)
  for file_name in test_list:
    print('-' * 70)
    print('Start to make classification use the data : {}'.format(file_name))
    print('-' * 70)
    # DA and oversampling data"
    df_train = loader.load_data(save_path, file_name, ratio, tech_name)
    if file_name == 'oversampling':
      df_train = loader.load_data(save_path, 'train', ratio, tech_name)
      eval_matrix(df_train, ROS_pipeline, 'macro', save_path)
    else:
      eval_matrix(df_train, original_pipeline, 'macro', save_path)


## Back_trans 

In [12]:

tvec = TfidfVectorizer(stop_words=None, max_features=1000000, ngram_range=(1, 3))
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
task_name = 'ag_news'
tech_name = 'Back_trans'
ratios = [0.2, 0.4, 0.6, 0.8, 1]
test_list = [
      'train',
      'oversampling',
      'downsampling',
      'aug_label_2',
      
      ]
clf = svm_clf
print('svm clf')
for ratio in ratios:
  print('*' * 50)
  print('ratio', ratio)
  
  execute(test_list, ratio, task_name, tech_name, tvec, clf)

svm clf
**************************************************
ratio 0.2
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9113
precision: 0.8829
recall: 0.8438
f1 score: 0.8593
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
value_counts 3    5112
1    5083
2    5050
0    5050
Name: label, dtype: int64
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9127
precision: 0.8794
recall: 0.8586
f1 score: 0.8677
----------------------------------------------------------------------
Start to make classification use the data : downsampling
-----------------------------------------

## BERT 

In [48]:


%%time

tvec = TfidfVectorizer(stop_words=None, max_features=1000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)

print('svm')

task_name = 'ag_news'
tech_name = 'Bert_ data augmentation-master'
clf = svm_clf
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  print('ratio', ratio)
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(unfine)_0.2',
      '3_rep_(unfine)_0.2',
      '4_rep_(unfine)_0.2',
      '5_rep_(unfine)_0.2',
      '6_rep_(unfine)_0.2']
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(unfine)_0.4',
      '3_rep_(unfine)_0.4',
      '4_rep_(unfine)_0.4',
      '5_rep_(unfine)_0.4',
      '6_rep_(unfine)_0.4']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(unfine)_0.6',
      '3_rep_(unfine)_0.6',
      '4_rep_(unfine)_0.6',
      '5_rep_(unfine)_0.6',
      '6_rep_(unfine)_0.6'] 
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(unfine)_0.8',
      '3_rep_(unfine)_0.8',
      '4_rep_(unfine)_0.8',
      '5_rep_(unfine)_0.8',
      '6_rep_(unfine)_0.8']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(unfine)_1',
      '3_rep_(unfine)_1',
      '4_rep_(unfine)_1',
      '5_rep_(unfine)_1', 
      '6_rep_(unfine)_1']

    print('<'*50)

  execute(test_list, ratio, task_name, tech_name, tvec, clf)




svm
ratio 0.2
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9113
precision: 0.8829
recall: 0.8438
f1 score: 0.8593
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
value_counts 3    5112
1    5083
2    5050
0    5050
Name: label, dtype: int64
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9127
precision: 0.8794
recall: 0.8586
f1 score: 0.8677
----------------------------------------------------------------------
Start to make classification use the data : downsampling
---------------------------------------------

## Fine-tuned BERT

In [53]:


%%time

tvec = TfidfVectorizer(stop_words=None, max_features=1000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
print('svm')

task_name = 'ag_news'
tech_name = 'Bert_ data augmentation-master'
clf = svm_clf
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(tuned)_0.2',
      '3_rep_(tuned)_0.2',
      '4_rep_(tuned)_0.2',
      '5_rep_(tuned)_0.2',
      '6_rep_(tuned)_0.2']
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(tuned)_0.4',
      '3_rep_(tuned)_0.4',
      '4_rep_(tuned)_0.4',
      '5_rep_(tuned)_0.4',
      '6_rep_(tuned)_0.4']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(tuned)_0.6',
      '3_rep_(tuned)_0.6',
      '4_rep_(tuned)_0.6',
      '5_rep_(tuned)_0.6',
      '6_rep_(tuned)_0.6']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(tuned)_0.8',
      '3_rep_(tuned)_0.8',
      '4_rep_(tuned)_0.8',
      '5_rep_(tuned)_0.8',
      '6_rep_(tuned)_0.8']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_(tuned)_1',
      '3_rep_(tuned)_1',
      '4_rep_(tuned)_1',
      '5_rep_(tuned)_1',
      '6_rep_(tuned)_1']

    print('<'*50)
    
  execute(test_list, ratio, task_name, tech_name, tvec, clf)




svm
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9113
precision: 0.8829
recall: 0.8438
f1 score: 0.8593
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
value_counts 3    5112
1    5083
2    5050
0    5050
Name: label, dtype: int64
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9127
precision: 0.8794
recall: 0.8586
f1 score: 0.8677
----------------------------------------------------------------------
Start to make classification use the data : downsampling
-------------------------------------------------------

## Fasttext

In [51]:

tvec = TfidfVectorizer(stop_words=None, max_features=1000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
clf = svm_clf
print('lr')

task_name = 'ag_news'
tech_name = 'Fasttext_augmentation'

ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  print('ratio', ratio)
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_0.2',
      '3_rep_0.2',
      '4_rep_0.2',
      '5_rep_0.2',
      '6_rep_0.2']
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_0.4',
      '3_rep_0.4',
      '4_rep_0.4',
      '5_rep_0.4',
      '6_rep_0.4']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_0.6',
      '3_rep_0.6',
      '4_rep_0.6',
      '5_rep_0.6',
      '6_rep_0.6']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_0.8',
      '3_rep_0.8',
      '4_rep_0.8',
      '5_rep_0.8',
      '6_rep_0.8']
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_rep_1',
      '3_rep_1',
      '4_rep_1',
      '5_rep_1',
      '6_rep_1']

    print('<'*50)


  
  execute(test_list, ratio, task_name, tech_name, tvec, clf)


lr
ratio 0.2
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9113
precision: 0.8829
recall: 0.8438
f1 score: 0.8593
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
value_counts 3    5112
1    5083
2    5050
0    5050
Name: label, dtype: int64
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.9127
precision: 0.8794
recall: 0.8586
f1 score: 0.8677
----------------------------------------------------------------------
Start to make classification use the data : downsampling
----------------------------------------------

## EDA

### SVM (test)

In [52]:
%%time
tvec = TfidfVectorizer(stop_words=None, max_features=1000000, ngram_range=(1, 3))

lr = LogisticRegression()
svm_clf = LinearSVC(random_state=0, tol=1e-5, dual=False)
clf = lr

task_name = 'ag_news'
tech_name = 'EDA'
ratios = [0.2, 0.4, 0.6, 0.8, 1]
for ratio in ratios:
  print('ratio', ratio)
  if ratio == 0.2:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_del_0.2',
      '3_del_0.2',
      '4_del_0.2',
      '5_del_0.2',
      '6_del_0.2',
      ]
    print('<'*50)
    # print('ratio', ratio)
  if ratio == 0.4:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_del_0.4',
      '3_del_0.4',
      '4_del_0.4',
      '5_del_0.4',
      '6_del_0.4',
      ]
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.6:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_del_0.6',
      '3_del_0.6',
      '4_del_0.6',
      '5_del_0.6',
      '6_del_0.6',
      ]
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 0.8:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_del_0.8',
      '3_del_0.8',
      '4_del_0.8',
      '5_del_0.8',
      '6_del_0.8',
      ]
    print('<'*50)
    # print('ratio', ratio)

  if ratio == 1:
    test_list = ['train',
                 'oversampling',
                 'downsampling',
                 '2_del_1',
      '3_del_1',
      '4_del_1',
      '5_del_1',
      '6_del_1',
      ]

    print('<'*50)
  execute(test_list, ratio, task_name, tech_name, tvec, clf)

ratio 0.2
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
----------------------------------------------------------------------
Start to make classification use the data : train
----------------------------------------------------------------------
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.8885
precision: 0.8699
recall: 0.7834
f1 score: 0.8053
----------------------------------------------------------------------
Start to make classification use the data : oversampling
----------------------------------------------------------------------
value_counts 3    5112
1    5083
2    5050
0    5050
Name: label, dtype: int64
value_counts 3    5112
1    5083
0    5050
2    1075
Name: label, dtype: int64
accuracy: 0.8964
precision: 0.8535
recall: 0.8539
f1 score: 0.8535
----------------------------------------------------------------------
Start to make classification use the data : downsampling
-------------------------------------------------